# Import

In [61]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [4]:
import load_script

# Loading the dataset

In [132]:
train_dataset,target=load_script.load_dataset(train=True)

In [133]:
target[target==-1]=0

# Defining the network

In [135]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(1,10,5)
        
        self.full1=nn.Linear(10 * 12 * 23,100)
        self.full2=nn.Linear(100,2)
        
    def forward(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x=x.view(-1)
        x=F.relu(self.full1(x))
        x=F.sigmoid(self.full2(x))
        
        return x

In [136]:
net=Net()
net

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (full1): Linear(in_features=2760, out_features=100, bias=True)
  (full2): Linear(in_features=100, out_features=2, bias=True)
)

# Training

In [146]:
net=Net()

optimizer = optim.SGD(net.parameters(), lr=0.0001)

for epoch in range(100):
    
    total_loss=0
    for train_element,target_element in zip(train_dataset,target):
        
        train_element=train_element.view(1,1,28,-1)
        
        optimizer.zero_grad()
        
        out=net(train_element)
        loss=criterion(out,target_element)
        loss.backward()
        optimizer.step()
        total_loss+=loss.data[0]
        
    print('Epoch:',epoch,'Loss:',total_loss)

Epoch: 0 Loss: 123.59187434690284
Epoch: 1 Loss: 110.5750338323694
Epoch: 2 Loss: 97.5880662583404
Epoch: 3 Loss: 89.4614565744123
Epoch: 4 Loss: 84.13598935109621
Epoch: 5 Loss: 80.45296291017439
Epoch: 6 Loss: 79.02461157373546
Epoch: 7 Loss: 76.95605606534264
Epoch: 8 Loss: 74.47921336911531
Epoch: 9 Loss: 71.02024788072595
Epoch: 10 Loss: 69.95454208943556
Epoch: 11 Loss: 68.11588730816074
Epoch: 12 Loss: 66.69582321416237
Epoch: 13 Loss: 64.19014810753288
Epoch: 14 Loss: 62.560875146562466
Epoch: 15 Loss: 60.34364521792304
Epoch: 16 Loss: 60.2726285789613
Epoch: 17 Loss: 58.84875081150676
Epoch: 18 Loss: 58.29148599417717
Epoch: 19 Loss: 56.64359336938651
Epoch: 20 Loss: 55.50077376281479
Epoch: 21 Loss: 54.69951479469819
Epoch: 22 Loss: 53.762793121670256
Epoch: 23 Loss: 51.99234382554641
Epoch: 24 Loss: 51.30305748965111
Epoch: 25 Loss: 50.34692552625165
Epoch: 26 Loss: 49.3883444787416
Epoch: 27 Loss: 48.316310491338754
Epoch: 28 Loss: 47.68122070021923
Epoch: 29 Loss: 46.63636

# Test

In [147]:
test_dataset,test_target=load_script.load_dataset(train=False)

In [148]:
test_target=test_target[:,1]>test_target[:,0]

In [149]:
test_dataset=test_dataset.view(100,1,28,50)

In [156]:
correct=0
for test_element,test_target_element in zip(test_dataset,test_target):
        
    test_element=test_element.view(1,1,28,-1)
    out=net(test_element)
    print(out)
    out=int(out[1]>out[0])
    if out==int(test_target_element):
        correct+=1
    
        

Variable containing:
 0.5127
 0.9620
[torch.FloatTensor of size 2]

Variable containing:
 0.9806
 0.0203
[torch.FloatTensor of size 2]

Variable containing:
 0.7791
 0.1063
[torch.FloatTensor of size 2]

Variable containing:
 0.5232
 0.4047
[torch.FloatTensor of size 2]

Variable containing:
 0.1681
 0.6564
[torch.FloatTensor of size 2]

Variable containing:
 0.9767
 0.1601
[torch.FloatTensor of size 2]

Variable containing:
 0.9183
 0.1065
[torch.FloatTensor of size 2]

Variable containing:
 0.9866
 0.0112
[torch.FloatTensor of size 2]

Variable containing:
 0.9150
 0.2268
[torch.FloatTensor of size 2]

Variable containing:
 0.9876
 0.0060
[torch.FloatTensor of size 2]

Variable containing:
 0.9963
 0.0148
[torch.FloatTensor of size 2]

Variable containing:
 0.3495
 0.5339
[torch.FloatTensor of size 2]

Variable containing:
 0.9054
 0.1725
[torch.FloatTensor of size 2]

Variable containing:
 0.1021
 0.9309
[torch.FloatTensor of size 2]

Variable containing:
 0.7442
 0.2783
[torch.Floa

In [157]:
correct

63